In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperConfig, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer
from datasets import load_dataset

/Users/sanchitgandhi/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load the original model
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

In [ ]:
# slice first 1/3 embeddings (=10 seconds input audio)
state_dict = model.state_dict()
state_dict["model.encoder.embed_positions.weight"] = state_dict["model.encoder.embed_positions.weight"][:500, :]

In [ ]:
# now load these weights back into the Whisper model, this time configured for this new seq len
config = WhisperConfig.from_pretrained("openai/whisper-tiny.en", max_source_positions=500)
model = WhisperForConditionalGeneration(config)

model.load_state_dict(state_dict)

In [ ]:
# load the tokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny.en")
# set the input length to 10 seconds
feature_extractor = WhisperFeatureExtractor(chunk_length=10)
# combine to form the processor
processor = WhisperProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
# check model works on a given sample
dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = dataset[0]["audio"]

In [ ]:
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features
pred_ids = model.generate(input_features, max_new_tokens=128)
pred_text = processor.batch_decode(pred_ids, skip_special_tokens=True)
print(pred_text)